In [1]:
import os
import dask
from dask.diagnostics import ProgressBar
import numpy as np
import my_funs 
import xarray as xr
import cProfile
import zarr
import matplotlib.pylab as plt
# dir = "/home/hamid/NASA_ABoVE/greeness/"
dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
# out_dir = "/home/hamid/NASA_ABoVE/greeness/working/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/working"
import folium
from folium.plugins import MousePosition


In [2]:
from dask.distributed import Client, progress
client = Client()

/data/home/hamiddashti/miniconda3/envs/geospatial/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40155 instead
  warnings.warn(
2022-08-29 15:31:32,847 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-2uz1nczy', purging
2022-08-29 15:31:32,847 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-t0wr6b17', purging
2022-08-29 15:31:32,848 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-eigk9ekp', purging
2022-08-29 15:31:32,848 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-3zz3ftbg', purging
2022-08-29 15:31:32,849 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-h1trn73c', purging
2022-08-29 15:31:32,849 - distrib

# Read downloaded data

In [3]:
ds_lai = xr.open_zarr(dir+'data/raw_data/noaa_cdr/lai_fapar/noaa_lai_fapar_clipped_raw.zarr')
lai = ds_lai["LAI"]
fapar= ds_lai["FAPAR"]
qa_lai = ds_lai["QA"]
ds_ndvi = xr.open_zarr(dir+'data/raw_data/noaa_cdr/ndvi/noaa_ndvi_clipped_raw.zarr')
ndvi = ds_ndvi["NDVI"]
qa_ndvi = ds_ndvi["QA"]

# Apply the quality control
we only apply the quality control to the LAI and the NDVI. Then we will select spectral bands (red, NIR) where we have good NDVI and calculate the EVI. 

In [4]:
# Convert QAs to binary
qa_lai_binary = my_funs.dec2bin(qa_lai,9)
qa_ndvi_binary = my_funs.dec2bin(qa_ndvi,16)

### Apply mask based on binary QAs aquired from previous step

In [5]:
qa_lai_binary = qa_lai_binary.astype(str)
qa_lai_mask = my_funs.avhrr_mask(qa_lai_binary,dask="allowed",var="LAI")
# lai_masked = lai.where(qa_mask)
fapar_maked = fapar.where(qa_lai_mask)

qa_ndvi_binary = qa_ndvi_binary.astype(str)
qa_mask_ndvi =my_funs.avhrr_mask(qa_ndvi_binary,dask="allowed",var="NDVI")
ndvi_masked = ndvi.where(qa_mask_ndvi)

In [6]:
# with ProgressBar():
#     lai_filtered = lai_masked.compute()
# with ProgressBar():
#     ndvi_filtered = ndvi_masked.compute()

# In case staritng the dask distributed client the above progressbar does not work and we should use the following commands
fapar_maked_ds = fapar_maked.to_dataset()
m = client.compute(fapar_maked_ds)

In [7]:
progress(m)

VBox()

In [9]:
fapar_maked_ds.to_zarr(out_dir+"data/processed_data/noaa_nc/lai_fapar/filtered/fapar_filtered.zarr")
# ndvi_filtered.to_netcdf(out_dir+"data/processed_data/noaa_nc/ndvi/filtered/ndvi_filtered.nc")

KeyboardInterrupt: 

Now lets read the LAI, NDVI and reflectance data and slect those NIR pixels we have good NDVI for them.

In [3]:
lai = xr.open_dataarray(dir+"data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered.nc")
ndvi = xr.open_dataarray(dir+"data/processed_data/noaa_nc/ndvi/filtered/ndvi_filtered.nc")
ds_refl = xr.open_zarr(dir+'data/raw_data/noaa_cdr/reflectance/noaa_reflectance_clipped_raw.zarr')
nir = ds_refl["SREFL_CH2"]

In [3]:
ndvi_good = my_funs.isfinite(ndvi)
nir_good = nir.where(ndvi_good)
nir_good.to_netcdf(dir+"data/processed_data/noaa_nc/reflectance/filtered/NIR_filtered.nc")

### Calculate the NIRv (ndvi*nir)

# Trend analyses

In [ ]:
lai_filtered = xr.open_dataarray(out_dir+"data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered.nc")
lai_growing = growing_season(lai_filtered)
lai_growing.to_netcdf(out_dir+"data/processed_data/noaa_nc/lai_fapar/grouped/lai_growing.nc")

In [ ]:
def xarray_Linear_trend(xarr, var_unit):
    from scipy import stats
    import numpy as np
    # getting shapes

    m = np.prod(xarr.shape[1:]).squeeze()
    n = xarr.shape[0]

    # creating x and y variables for linear regression
    x = xarr.time.to_pandas().index.to_julian_date().values[:, None]
    y = xarr.to_masked_array().reshape(n, -1)

    # ############################ #
    # LINEAR REGRESSION DONE BELOW #
    xm = x.mean(0)  # mean
    ym = y.mean(0)  # mean
    ya = y - ym  # anomaly
    xa = x - xm  # anomaly

    # variance and covariances
    xss = (xa**2).sum(0) / (n - 1)  # variance of x (with df as n-1)
    yss = (ya**2).sum(0) / (n - 1)  # variance of y (with df as n-1)
    xys = (xa * ya).sum(0) / (n - 1)  # covariance (with df as n-1)
    # slope and intercept
    slope = xys / xss
    intercept = ym - (slope * xm)
    # statistics about fit
    df = n - 2
    r = xys / (xss * yss)**0.5
    t = r * (df / ((1 - r) * (1 + r)))**0.5
    p = stats.distributions.t.sf(abs(t), df)

    # misclaneous additional functions
    # yhat = dot(x, slope[None]) + intercept
    # sse = ((yhat - y)**2).sum(0) / (n - 2)  # n-2 is df
    # se = ((1 - r**2) * yss / xss / df)**0.5

    # preparing outputs
    out = xarr[:2].mean('time')
    # first create variable for slope and adjust meta
    xarr_slope = out.copy()
    xarr_slope.name = '_slope'
    xarr_slope.attrs['units'] = var_unit
    xarr_slope.values = slope.reshape(xarr.shape[1:])
    # do the same for the p value
    xarr_p = out.copy()
    xarr_p.name = '_Pvalue'
    xarr_p.attrs[
        'info'] = "If p < 0.05 then the results from 'slope' are significant."
    xarr_p.values = p.reshape(xarr.shape[1:])
    # join these variables
    xarr_out = xarr_slope.to_dataset(name='slope')
    xarr_out['pval'] = xarr_p

    return xarr_out

In [ ]:
lai_growing = lai_growing.rename({"year":"time"})

In [ ]:
import pandas as pd
a = lai_growing.time.to_pandas().index
b = pd.to_datetime(a, format='%Y')
# lai_growing.time = b
lai_growing["time"] = b
lai_growing_trend = xarray_trend(lai_growing, var_unit="m / m / year")

In [ ]:
plt.figure(figsize=(16,8))
lai_growing_trend["slope"].plot()
plt.savefig(out_dir+"/Figures/LAI_growing_trend.png")

In [ ]:
lai_growing_mean = lai_growing.mean(["latitude","longitude"])
lai_growing_mean